In [ ]:
import pandas as pd
import numpy as np


import neutcurve

from plotnine import *

import csv
import matplotlib.pyplot as plt

In [ ]:
CBP = neutcurve.colorschemes.CBPALETTE
CBM = neutcurve.colorschemes.CBMARKERS
theme_set(theme_seaborn(style='white', context='paper', font_scale=1))

In [ ]:
df = pd.read_csv('./deltaCT_fract_infectivity.csv')
df['serum'] = df['serum'].astype(str)
fill = {'serum': ['15529', '15529', '15529', '15529', '15529'],
        'virus': ['FullLength_2', 'delta18', 'delta18', 'delta18', 'delta18'],
        'replicate': [2, 2, 2, 2, 2],
        'concentration': [df.iloc[18]['concentration'], df.iloc[24]['concentration'],
                          df.iloc[26]['concentration'], df.iloc[28]['concentration'], df.iloc[29]['concentration']],
        'fraction infectivity': [0.904158, 0.073803, 0.614360, 0.927584, 1.017870]}
fill_df = pd.DataFrame.from_dict(fill)
df_fillrep2 = pd.concat([df, fill_df], ignore_index=True)
display(df_fillrep2)

In [ ]:
df_fillrep2['serum'] = df_fillrep2['serum'].replace('15529', 'SARS-CoV-2+, 43 days p.s.o.')
df_fillrep2['virus'] = df_fillrep2['virus'].replace('FullLength_3', 'Full Length')
display(df_fillrep2)

In [ ]:
final_fits = neutcurve.CurveFits(df_fillrep2, fixtop=True)

In [ ]:
serum = 'SARS-CoV-2+, 43 days p.s.o.'

In [ ]:
curve = final_fits.getCurve(serum=serum, virus='FullLength_1', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
print(f"This is a dilution of 1:{1/curve.ic50():.4g}")
final_fl1_ic50 = curve.ic50()
fig, ax = curve.plot()

In [ ]:
curve = final_fits.getCurve(serum=serum, virus='FullLength_2', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
print(f"This is a dilution of 1:{1/curve.ic50():.4g}")
final_fl2_ic50 = curve.ic50()
fig, ax = curve.plot()

In [ ]:
curve = final_fits.getCurve(serum=serum, virus='Full Length', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
print(f"This is a dilution of 1:{1/curve.ic50():.4g}")
final_fl3_ic50 = curve.ic50()
fig, ax = curve.plot()

In [ ]:
curve = final_fits.getCurve(serum=serum, virus='delta18', replicate='average')
print(f"The IC50 is {curve.ic50():.3g}")
print(f"This is a dilution of 1:{1/curve.ic50():.4g}")
final_delta18_ic50 = curve.ic50()
fig, ax = curve.plot()

In [ ]:
curve = final_fits.getCurve(serum=serum, virus='delta21', replicate='1')
print(f"The IC50 is {curve.ic50():.3g}")
print(f"This is a dilution of 1:{1/curve.ic50():.4g}")
final_delta21_ic50 = curve.ic50()
fig, ax = curve.plot()

In [ ]:
fig, axes = final_fits.plotSera(xlabel = 'serum dilution')

In [ ]:
fig, axes = final_fits.plotSera(viruses=['Full Length', 'delta18', 'delta21'], xlabel = 'serum dilution')
fig.savefig('./200611_detaCT_neut.png', bbox_inches='tight')

In [ ]:
print(f"FL_1 (DK neuts), 1:{1/final_fl1_ic50:.5}\nFL_2 (CT test), 1:{1/final_fl2_ic50:.5}\nFL_3 (re-run), 1:{1/final_fl3_ic50:.5}")
print(f"delta18, 1:{1/final_delta18_ic50:.5}\ndelta21, 1:{1/final_delta21_ic50:.5}")

# Titer Data

Flow cytometry and luciferase titering data for viruses made with full length, or deltaCT Spikes.

In [ ]:
luctiters = pd.read_csv('./200611_luctiters.csv')

VEP_list = ['Full Length', 'delta18', 'delta21', 'None', 'VSV G']
VEP_order = ['Full Length', 'delta18', 'delta21', 'VSV G', 'None']
luctiters['VEP'] = VEP_list
luctiters['VEP_order'] = pd.Categorical(luctiters['VEP'], categories = VEP_order)
luctiters['Plot_Titer'] = [luctiters.iloc[0]['Avg_RLUpermL-neut_plate'], luctiters.iloc[1]['Avg_RLUpermL-neut_plate'],
                          luctiters.iloc[2]['Avg_RLUpermL-neut_plate'], luctiters.iloc[3]['Avg_RLUpermL-neut_plate'],
                          luctiters.iloc[4]['Avg_RLUpermL-titer_plate']]

display(luctiters)

In [ ]:
RLU_titers_plot = (ggplot(luctiters, aes(x='VEP_order', y='Plot_Titer')) +
                         geom_point(size=3, position=position_dodge(width=0.2, preserve='total')) +
#                           scale_fill_manual(values=CBPALETTE[1:3], name='Backbone') +
#                           scale_shape_manual(values=['o'], name='Backbone') +
                          theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5, size=12),
                                axis_title_y = element_text(size=11),
                                plot_title=element_text(size=13),
                                figure_size=(4, 2.5)) +
                          scale_y_log10(limits=[1e2, 1e10], breaks=[1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10]) +
                          #geom_hline(yintercept=10) +
                          ylab('relative luciferase units per mL') +
                          xlab('') +
                          ggtitle('Luciferase-IRES-ZsGreen backbone'))

_ = RLU_titers_plot.draw()

RLU_titers_plot.save('200611_luciferase_titers.pdf')

In [ ]:
flow_titers = pd.read_csv('./200605_deltaCT_flowtiter.csv')
flow_titers_toplot = flow_titers[flow_titers['Sample_Cat'] == 'HDM+25']
display(flow_titers_toplot)

In [ ]:
final_flow_titers = {'VEP': ['None', 'IDT_EK', 'delta18', 'delta21', 'VSVG'],
                     'Avg Titer': [((flow_titers_toplot.loc[0]['Titer'] + flow_titers_toplot.loc[7]['Titer'])/2),
                                   ((flow_titers_toplot.loc[1]['Titer'] + flow_titers_toplot.loc[8]['Titer'])/2),
                                   ((flow_titers_toplot.loc[2]['Titer'] + flow_titers_toplot.loc[9]['Titer'])/2),
                                   ((flow_titers_toplot.loc[3]['Titer'] + flow_titers_toplot.loc[10]['Titer'] + flow_titers_toplot.loc[21]['Titer'])/3),
                                   ((flow_titers_toplot.loc[4]['Titer'] + flow_titers_toplot.loc[11]['Titer'])/2)]                                   
                     }
final_titers_df = pd.DataFrame.from_dict(final_flow_titers)
flow_VEP_list = ['None', 'Full Length', 'delta18', 'delta21', 'VSV G']
flow_VEP_order = ['Full Length', 'delta18', 'delta21', 'VSV G', 'None']
final_titers_df['plot_names'] = flow_VEP_list
final_titers_df['VEP_order'] = pd.Categorical(final_titers_df['plot_names'], categories = flow_VEP_order)

display(final_titers_df)


In [ ]:
figure_spike_titers = (ggplot(final_titers_df.replace(0, float('NaN')),
                              aes(x='VEP_order', y='Avg Titer')
                              ) +
                              geom_point(size=3, position=position_dodge(width=0.2, preserve='total')) +
                              geom_text(label='n.d.', x=5, y=1, size=10, fontweight='normal', fontstyle='normal') +
                              theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5, size=12),
                                axis_title_y = element_text(size=11),
                                plot_title=element_text(size=13),
                                figure_size=(4, 2.5)) +
                              scale_y_log10(limits=[1e1, 1e7], breaks=[1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]) +
                              #geom_hline(yintercept=10) +
                              ylab('transduction units per mL') +
                              xlab('') +
                              ggtitle('ZsGreen backbone titers from flow cytometry') 
                             )
_ = figure_spike_titers.draw()

figure_spike_titers.save('200611_ZsGreen_titers.pdf')